# crop_square_centered_on_qrcode

This notebook performs the following steps:
* detects QRcode and calculates its centroid
* extracts a square from the image centered on the QRcode

In [7]:
from pyzbar.pyzbar import decode
import cv2
import glob
import os
import papermill
import pandas as pd
import numpy as np
import math

In [8]:
# This cell is tagged as "parameters" to enable use of papermill

DATADIR = '/home/aubrey/Desktop/gpepp_nursery'
CROP_WIDTH = 2500

In [9]:
# papermill.inspect_notebook('crop_square_centered_on_qrcode.ipynb')

In [10]:
def crop_square_centered_on_qrcode(img, crop_width):
    barcode = decode(img)[0]
    rect = barcode.rect
    x_center = rect.left + (rect.width // 2)
    y_center = rect.top + (rect.height // 2)

    half_width = crop_width // 2
    start_row = y_center - half_width
    end_row = y_center + half_width
    start_col = x_center - half_width
    end_col = x_center + half_width
    # If crop_width is even, we add an extra row and column
    if (crop_width % 2):
        end_row += 1
        end_col += 1         
    return img[start_row:end_row, start_col:end_col]

In [11]:
def barcode_size(barcode):
    """
    Returns the average length of the 4 sides of the barcode polygon
    Requires:
    import numpy as np
    import math
    """
    points = barcode.polygon
    # Repeat the first point at the end of the list to complete the polygon
    points.append(points[0])
    lengths = []
    for i in range(4):
        lengths.append(math.dist(points[i], points[i+1]))    
    return np.mean(lengths)
    
# img = cv2.imread('/home/aubrey/Desktop/gpepp_nursery/31/cropped.png')
# barcode = decode(img)[0]
# barcode_size(barcode)

In [12]:
# MAIN

df_imagelist = pd.read_csv(f'{DATADIR}/imagelist.csv')

for i, r in df_imagelist.iterrows():
    sn = os.path.basename(r.filename).replace('.png', '')
    original_image_path = f'{DATADIR}/{sn}/original.png'
    cropped_image_path =  f'{DATADIR}/{sn}/cropped.png'
    print(f'Processing{original_image_path}')
    img = cv2.imread(original_image_path)
    cropped_img = crop_square_centered_on_qrcode(img, CROP_WIDTH) 
    cv2.imwrite(cropped_image_path, cropped_img)

print('widths of barcodes')
for f in glob.glob('/home/aubrey/Desktop/gpepp_nursery/*/cropped.png'):
    img = cv2.imread(f)
    barcode = decode(img)[0]
    bc_size = barcode_size(barcode) 
    print(f'{f} {bc_size:.1f}')
    
print('Finished.')    

Processing/home/aubrey/Desktop/gpepp_nursery/31/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/26/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/28/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/27/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/38/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/35/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/33/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/32/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/30/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/25/original.png
Processing/home/aubrey/Desktop/gpepp_nursery/29/original.png
widths of barcodes
/home/aubrey/Desktop/gpepp_nursery/28/cropped.png 216.4
/home/aubrey/Desktop/gpepp_nursery/30/cropped.png 216.2
/home/aubrey/Desktop/gpepp_nursery/29/cropped.png 216.5
/home/aubrey/Desktop/gpepp_nursery/32/cropped.png 216.3
/home/aubrey/Desktop/gpepp_nursery/38/cropped.png 217.4
/home/aubrey/Desktop/gpepp_nur